<a href="https://colab.research.google.com/github/iraidaantropova/machine_L/blob/main/HW2_SinitsaI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Домашнее задание

*Самостоятельно повторить tfidf (документация https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана.

Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог)
Повторить п.2, но используя уже не медиану, а max

*Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (взяв список новостей пользователя)
подсказка 1: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал
подсказка 2: нужен именно idf, как вес.

Сформировать на выходе единую таблицу, сравнивающую качество 2/3 разных метода получения эмбедингов пользователей: median, max, idf_mean по метрикам roc_auc, precision, recall, f_score

Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

In [87]:
import pandas as pd

In [88]:
# Наши новости

!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR" -O articles.csv && rm -rf /tmp/cookies.txt

--2023-06-15 07:39:14--  https://docs.google.com/uc?export=download&confirm=t&id=1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR
Resolving docs.google.com (docs.google.com)... 108.177.13.102, 108.177.13.138, 108.177.13.101, ...
Connecting to docs.google.com (docs.google.com)|108.177.13.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/n1pi1noe9j9q5ccda7th00rmh1cn5dk3/1686814725000/14904333240138417226/*/1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR?e=download&uuid=7bfa6a36-35e4-4632-a27d-2235fb5ba28f [following]
--2023-06-15 07:39:14--  https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/n1pi1noe9j9q5ccda7th00rmh1cn5dk3/1686814725000/14904333240138417226/*/1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR?e=download&uuid=7bfa6a36-35e4-4632-a27d-2235fb5ba28f
Resolving doc-0s-c0-docs.googleusercontent.com (doc-0s-c0-docs.googleusercontent.com)... 173.194.212.132,

In [89]:
news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


In [90]:
# Загружаю пользователей и списки последних прочитанных новостей
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [91]:
!pip install razdel pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [92]:
# предобработка текстов
import re
import numpy as np
from gensim.corpora.dictionary import Dictionary
from razdel import tokenize
import pymorphy2  # Морфологический анализатор

In [93]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [94]:
stopword_ru = stopwords.words('russian')
print(len(stopword_ru))

151


In [95]:
!wget 'https://drive.google.com/uc?export=download&id=17wVn5YPpMjHToctGgff_KfSeWcIIlf7c' -O stopwords.txt

--2023-06-15 07:39:23--  https://drive.google.com/uc?export=download&id=17wVn5YPpMjHToctGgff_KfSeWcIIlf7c
Resolving drive.google.com (drive.google.com)... 74.125.141.113, 74.125.141.101, 74.125.141.102, ...
Connecting to drive.google.com (drive.google.com)|74.125.141.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/c22e3070t11o5t775coil10gppla118t/1686814725000/14904333240138417226/*/17wVn5YPpMjHToctGgff_KfSeWcIIlf7c?e=download&uuid=318a544b-4217-46d9-a09a-02d62c884763 [following]
--2023-06-15 07:39:24--  https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/c22e3070t11o5t775coil10gppla118t/1686814725000/14904333240138417226/*/17wVn5YPpMjHToctGgff_KfSeWcIIlf7c?e=download&uuid=318a544b-4217-46d9-a09a-02d62c884763
Resolving doc-0s-c0-docs.googleusercontent.com (doc-0s-c0-docs.googleusercontent.com)... 173.194.212.132, 2607

In [96]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]

stopword_ru += additional_stopwords
len(stopword_ru)

776

In [97]:
def clean_text(text):
    '''
    очистка текста

    на выходе очищеный текст
    '''
    if not isinstance(text, str):
        text = str(text)

    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    text = re.sub('n', ' ', text)

    return text

cache = {}
morph = pymorphy2.MorphAnalyzer()

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист лемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)

    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w) > 1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)

    words_lem_without_stopwords = [i for i in words_lem if not i in stopword_ru] # [6]

    return words_lem_without_stopwords

In [98]:
%%time
from tqdm import tqdm
tqdm.pandas()

# Очистка текста
news['title'] = news['title'].progress_apply(lambda x: clean_text(x))

100%|██████████| 27000/27000 [00:55<00:00, 483.15it/s] 

CPU times: user 43 s, sys: 696 ms, total: 43.7 s
Wall time: 55.9 s


In [99]:
news['title'].iloc[:10]

0    заместитель председателя правительства рф серг...
1    матч  финала кубка россии по футболу был приос...
2    форвард авангарда томаш заборский прокомментир...
3    главный тренер кубани юрий красножан прокоммен...
4    решением попечительского совета владивостокско...
5    ио главного тренера вячеслав буцаев прокоммент...
6    запорожский металлург дома потерпел разгромное...
7    сборная сша одержала победу над австрией со сч...
8    бывший защитник сборной россии дарюс каспарайт...
9    полузащитник цска зоран тошич после победы над...
Name: title, dtype: object

In [100]:
news['title'].iloc[:2].apply(lambda x: lemmatization(x))

0    [заместитель, председатель, правительство, рф,...
1    [матч, финал, кубок, россия, футбол, приостано...
Name: title, dtype: object

In [101]:
%%time
# Лемматизация текста
news['title'] = news['title'].progress_apply(lambda x: lemmatization(x))

100%|██████████| 27000/27000 [04:31<00:00, 99.54it/s] 

CPU times: user 4min 16s, sys: 1.2 s, total: 4min 17s
Wall time: 4min 31s


In [102]:
#Обучение модели
#формирование списка текстов
texts = list(news['title'].values)

# Создание корпуса из списка с текстами
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [103]:
#Запуск обучения

N_topic = 20


In [104]:
%%time
from gensim.models import LdaModel

# Обучаем модель на корпусе
lda = LdaModel(common_corpus, num_topics=N_topic, id2word=common_dictionary, passes=2)

CPU times: user 1min 30s, sys: 42.2 s, total: 2min 12s
Wall time: 1min 30s


In [105]:
from gensim.test.utils import datapath

# Сохраняем модель на диск
temp_file = datapath("model.lda")
lda.save(temp_file)

In [106]:
# Загружаю обученную модель с диска
lda = LdaModel.load(temp_file)

In [107]:
# Создаю новый корпус документов
other_texts = list(news['title'].iloc[:3])
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc]

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'играть', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'хороший']


[(0, 0.03218093),
 (2, 0.081980065),
 (3, 0.63120776),
 (6, 0.098055534),
 (9, 0.050582193),
 (18, 0.08753586)]

In [108]:
#модель обучена. Решаю 2 вопроса: -как выглядят темы; как получить для документа вектор значений (вероятности принадлежности каждой теме)

x = lda.show_topics(num_topics=N_topic, num_words=15, formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

# Печатю только слова
for topic, words in topics_words:
    print(f"topic_{topic}: " + " ".join(words))

topic_0: германия польша маршрут польский остаток парк брюссель дания компенсировать взорваться консультация венгрия австрия поляк немецкий
topic_1: произойти тело взрыв место дом аэропорт причина жертва изз местный статья век удаться египет рейс
topic_2: банк новый санкция налог якобы расчёт океан падение риск курс доллар операция конец соглашение следствие
topic_3: миссия сайт мероприятие команда узнать увидеть участник состояться игра хороший открытие гость организатор известный проект
topic_4: испытание академия оборудование индия грант внедрение горизонт лесной русь паника потенциально случайный лётчик вырабатывать комплексный
topic_5: сша россия российский американский ракета сторона район запуск данные сила государство граница территория помощь действие
topic_6: смерть температура рак проверка россия земля космос восток день дональд товар значительно подчёркивать риск погода
topic_7: всё первый научный день жизнь большой земля украина очень новый россия газета эксперт последний 

In [109]:
#Функция, которая возвращает векторное представление новости
def get_lda_vector(lda, text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]

    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(N_topic):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [110]:
get_lda_vector(lda, news['title'].iloc[0])

array([0.        , 0.        , 0.        , 0.10570618, 0.        ,
       0.        , 0.        , 0.        , 0.25273407, 0.        ,
       0.        , 0.        , 0.        , 0.46032831, 0.17353272,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [111]:
%%time
topic_matrix = pd.DataFrame([get_lda_vector(lda, text) for text in news['title'].values])
topic_matrix.columns = [f'topic_{i}' for i in range(N_topic)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+[f'topic_{i}' for i in range(N_topic)]]
topic_matrix.head(5)

CPU times: user 39 s, sys: 19.4 s, total: 58.3 s
Wall time: 40.3 s


,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,6,0.000000,0.000000,0.000000,0.105707,0.0,0.0,0.000000,0.000000,0.252684,...,0.000000,0.000000,0.0,0.46033,0.173580,0.000000,0.0,0.0,0.000000,0.0
1,4896,0.000000,0.590311,0.000000,0.385994,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.0
2,4897,0.032181,0.000000,0.082011,0.631209,0.0,0.0,0.098037,0.000000,0.000000,...,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.087534,0.0
3,4898,0.000000,0.000000,0.000000,0.285477,0.0,0.0,0.015330,0.449658,0.059021,...,0.000000,0.071078,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.0
4,4899,0.000000,0.000000,0.000000,0.186120,0.0,0.0,0.000000,0.207393,0.095432,...,0.208042,0.000000,0.0,0.00000,0.256559,0.025795,0.0,0.0,0.000000,0.0


In [112]:
#векторные представления пользователей
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [113]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[[f'topic_{i}' for i in range(N_topic)]].values))

In [114]:
doc_dict[293672]

array([0.        , 0.        , 0.12030601, 0.09205492, 0.02933887,
       0.        , 0.        , 0.        , 0.40104607, 0.        ,
       0.        , 0.11661332, 0.12348881, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.09908946, 0.        ])

In [115]:
def get_user_embedding(user_articles_list, doc_dict, func):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = func(user_vector, axis=0)
    return user_vector

In [116]:
user_articles_list = users['articles'].iloc[33]

get_user_embedding(user_articles_list, doc_dict, np.mean)

array([0.        , 0.00957184, 0.04811623, 0.        , 0.01401632,
       0.13588319, 0.00549807, 0.19568975, 0.22290835, 0.01094971,
       0.04829771, 0.        , 0.00240416, 0.03300292, 0.10535923,
       0.04087642, 0.02971555, 0.00922114, 0.02302858, 0.05220904])

In [117]:
FUNC = np.mean

In [118]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,u105138,0.000000,0.049034,0.052484,0.118389,0.007750,0.029148,0.000000,0.168780,0.141776,...,0.005414,0.037916,0.024904,0.051419,0.100325,0.008500,0.065580,0.063292,0.016515,0.034947
1,u108690,0.001735,0.025882,0.031512,0.020269,0.016128,0.128217,0.049059,0.305667,0.092820,...,0.031623,0.013625,0.000000,0.037833,0.139038,0.012886,0.038483,0.008250,0.028602,0.000000
2,u108339,0.000000,0.102906,0.025225,0.017815,0.013974,0.152371,0.085944,0.121897,0.068039,...,0.029855,0.009644,0.000000,0.039250,0.155578,0.003129,0.036956,0.070208,0.026247,0.019912


In [119]:
!wget 'https://drive.google.com/uc?export=download&id=1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK' -O users_churn.csv

--2023-06-15 07:47:20--  https://drive.google.com/uc?export=download&id=1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK
Resolving drive.google.com (drive.google.com)... 173.194.215.138, 173.194.215.101, 173.194.215.139, ...
Connecting to drive.google.com (drive.google.com)|173.194.215.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0k-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ukeuba5j785f5kilr4rgm8trp91ajheg/1686815175000/14904333240138417226/*/1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK?e=download&uuid=3c7ae75d-d5af-4489-90ce-7cbce31a33cc [following]
--2023-06-15 07:47:21--  https://doc-0k-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ukeuba5j785f5kilr4rgm8trp91ajheg/1686815175000/14904333240138417226/*/1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK?e=download&uuid=3c7ae75d-d5af-4489-90ce-7cbce31a33cc
Resolving doc-0k-c0-docs.googleusercontent.com (doc-0k-c0-docs.googleusercontent.com)... 173.194.212.132, 

In [120]:
target = pd.read_csv("users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [121]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,churn
0,u105138,0.000000,0.049034,0.052484,0.118389,0.007750,0.029148,0.000000,0.168780,0.141776,...,0.037916,0.024904,0.051419,0.100325,0.008500,0.065580,0.063292,0.016515,0.034947,0
1,u108690,0.001735,0.025882,0.031512,0.020269,0.016128,0.128217,0.049059,0.305667,0.092820,...,0.013625,0.000000,0.037833,0.139038,0.012886,0.038483,0.008250,0.028602,0.000000,1
2,u108339,0.000000,0.102906,0.025225,0.017815,0.013974,0.152371,0.085944,0.121897,0.068039,...,0.009644,0.000000,0.039250,0.155578,0.003129,0.036956,0.070208,0.026247,0.019912,1


In [122]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [123]:
# разделение данных на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]],
                                                    X['churn'], random_state=0)

In [124]:
model = LogisticRegression()
# обучение
model.fit(X_train, y_train)

LogisticRegression()

In [125]:
# прогноз для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:20]

array([0.05721111, 0.33195147, 0.51313209, 0.04476055, 0.04326843,
       0.14859181, 0.04376675, 0.00545681, 0.03618676, 0.05277994,
       0.05879825, 0.22561147, 0.00855423, 0.13122446, 0.00886733,
       0.24800974, 0.0474097 , 0.00332657, 0.00091812, 0.2839678 ])

In [126]:
metrics_df = pd.DataFrame(columns=['model', 'thresh', 'F-Score', 'Precision', 'Recall', 'ROC AUC'])
metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC


In [127]:
from sklearn.metrics import (f1_score, roc_auc_score, precision_score,
                             classification_report, precision_recall_curve, confusion_matrix)

In [128]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.2977684527266521, F-Score=0.734, Precision=0.710, Recall=0.759


In [129]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.964409558695273

In [130]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

<ipython-input-130-67fa86a82b10>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.297768,0.733728,0.709924,0.759184,0.96441


In [131]:
FUNC = np.median

In [132]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,u105138,0.0,0.031856,0.026485,0.12613,0.0,0.000000,0.000000,0.206684,0.071165,...,0.000000,0.025905,0.0,0.030079,0.041822,0.0,0.051431,0.034490,0.000000,0.012483
1,u108690,0.0,0.006477,0.034778,0.00000,0.0,0.110521,0.042021,0.294553,0.102530,...,0.006806,0.000000,0.0,0.013019,0.126172,0.0,0.000000,0.000000,0.006342,0.000000
2,u108339,0.0,0.088633,0.000000,0.00000,0.0,0.129707,0.070319,0.120205,0.059543,...,0.007271,0.000000,0.0,0.037882,0.180323,0.0,0.037404,0.077191,0.007591,0.012695


In [133]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,churn
0,u105138,0.0,0.031856,0.026485,0.12613,0.0,0.000000,0.000000,0.206684,0.071165,...,0.025905,0.0,0.030079,0.041822,0.0,0.051431,0.034490,0.000000,0.012483,0
1,u108690,0.0,0.006477,0.034778,0.00000,0.0,0.110521,0.042021,0.294553,0.102530,...,0.000000,0.0,0.013019,0.126172,0.0,0.000000,0.000000,0.006342,0.000000,1
2,u108339,0.0,0.088633,0.000000,0.00000,0.0,0.129707,0.070319,0.120205,0.059543,...,0.000000,0.0,0.037882,0.180323,0.0,0.037404,0.077191,0.007591,0.012695,1


In [134]:
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]],
                                                    X['churn'], random_state=0)

In [135]:
model = LogisticRegression()
# обучение
model.fit(X_train, y_train)

LogisticRegression()

In [136]:
# прогноз для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:20]

array([5.90229029e-02, 1.61998981e-01, 5.19510642e-01, 5.77566574e-02,
       2.56960894e-02, 3.19051831e-02, 6.28437039e-02, 3.65768981e-03,
       4.42803145e-02, 3.45015361e-02, 1.72508577e-02, 3.17818141e-02,
       4.14706272e-03, 7.44609862e-02, 3.38015695e-03, 3.35429935e-01,
       3.72569576e-02, 3.03840708e-03, 4.52196879e-04, 4.15019313e-01])

In [137]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.26472442223112347, F-Score=0.767, Precision=0.711, Recall=0.833


In [138]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9718774347345775

In [139]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

<ipython-input-139-67fa86a82b10>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.297768,0.733728,0.709924,0.759184,0.964410
1,median,0.264724,0.766917,0.710801,0.832653,0.971877


In [140]:
FUNC = np.max

In [141]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,u105138,0.000000,0.148556,0.141628,0.290281,0.029339,0.118149,0.000000,0.327896,0.401046,...,0.032482,0.116613,0.123489,0.150963,0.357715,0.050998,0.177030,0.229538,0.099089,0.114343
1,u108690,0.010408,0.110079,0.064123,0.102790,0.096767,0.296759,0.128598,0.504949,0.156669,...,0.116265,0.081752,0.000000,0.150451,0.315763,0.077316,0.182061,0.033817,0.123924,0.000000
2,u108339,0.000000,0.290683,0.121130,0.106888,0.083844,0.353428,0.202464,0.165204,0.142373,...,0.090984,0.040686,0.000000,0.084254,0.248392,0.018774,0.073960,0.122944,0.127048,0.063782


In [142]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,churn
0,u105138,0.000000,0.148556,0.141628,0.290281,0.029339,0.118149,0.000000,0.327896,0.401046,...,0.116613,0.123489,0.150963,0.357715,0.050998,0.177030,0.229538,0.099089,0.114343,0
1,u108690,0.010408,0.110079,0.064123,0.102790,0.096767,0.296759,0.128598,0.504949,0.156669,...,0.081752,0.000000,0.150451,0.315763,0.077316,0.182061,0.033817,0.123924,0.000000,1
2,u108339,0.000000,0.290683,0.121130,0.106888,0.083844,0.353428,0.202464,0.165204,0.142373,...,0.040686,0.000000,0.084254,0.248392,0.018774,0.073960,0.122944,0.127048,0.063782,1


In [143]:
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]],
                                                    X['churn'], random_state=0)

In [144]:
model = LogisticRegression()
# обучение
model.fit(X_train, y_train)

LogisticRegression()

In [145]:
# прогноз для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:20]

array([1.98628642e-02, 5.60712956e-02, 7.94819325e-01, 1.10343153e-02,
       1.04782061e-02, 3.38642210e-02, 1.51718496e-02, 5.60273243e-02,
       2.22848190e-02, 1.12332954e-01, 2.09580277e-02, 1.54034462e-02,
       1.52916909e-02, 6.99414841e-02, 5.74281592e-03, 5.27950381e-01,
       2.70327454e-02, 6.48204482e-06, 1.95677266e-03, 1.66173219e-01])

In [146]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.3484374049856208, F-Score=0.792, Precision=0.762, Recall=0.824


In [147]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9732310018024304

In [148]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

<ipython-input-148-67fa86a82b10>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.297768,0.733728,0.709924,0.759184,0.964410
1,median,0.264724,0.766917,0.710801,0.832653,0.971877
2,amax,0.348437,0.792157,0.762264,0.824490,0.973231


In [149]:
metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.297768,0.733728,0.709924,0.759184,0.964410
1,median,0.264724,0.766917,0.710801,0.832653,0.971877
2,amax,0.348437,0.792157,0.762264,0.824490,0.973231
